In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, PreTrainedTokenizerFast, BartForConditionalGeneration
from datasets import Dataset, load_dataset
from torch.utils.data import DataLoader

import torch
import pandas as pd
import numpy as np
import time
import sys
import requests
import json

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 데이터 로드
### 삼성전자 관련된 하나의 토픽에 대한 30개 기사(혜진님이 슬랙에 올려주신 파일)

In [15]:
article30 = pd.read_csv('result30.csv', index_col=0)

In [4]:
def preprocessing(text):
    text = text.replace('<b>', '')
    text = text.replace('</b>', '')
    text = text.replace('&apos;', '')
    text = text.replace('...', '')
    text = text.replace('\u200b', '')
    text = text.replace('M&amp;A', '')
    text = text.replace('&quot;', '')
    text = text.replace('\xa0', '')
    text = text.strip()
    return text

In [19]:
add_title_context = []
for _, t in article30.iterrows():
    tem = [c for c in t['context'].split('.') if len(c) > 3]
    try:
        con = t['title'] + ' ' + tem[0] + '. ' + tem[1] + '.'
    except:
        con = t['title'] + ' ' + tem[0] + '.'
    con = preprocessing(con)
    add_title_context.append(con)

title_context = [preprocessing(con) for con in article30.title]

## Test 1
### 제목 + 본문 앞 2문장(description말고 context)

30개 기사를 (제목 + 앞 1,2문장) 합친 후, 30개 내용을 하나로 합쳐서 요약.

### KoT5

In [20]:
KoT5_tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
KoT5 = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")
KoT5.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [21]:
# 합친 내용이 너무 긴 경우에는 에러 발생할 수 있음
s = time.time()
input_ids = KoT5_tokenizer.encode('.'.join(add_title_context))
print('input_ids length: ', len(input_ids))
summary_ids = KoT5.generate(torch.tensor([input_ids]).to(device),  num_beams=4,  max_length=512,  eos_token_id=1)
print(KoT5_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))
print(time.time() - s)

input_ids length:  4002
삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 13일부터 시행하여 스마트폰 카메라의 사용성을 한 단계 더 높이고 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈2 프로와 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 통해 360 오디오 레코딩 기능을 지원한다.
4.22922682762146


- 요약문 품질은 나쁘지 않지만, 한 문장에 비해 너무 길게 출력되고, 시간이 오래 걸림.

## Kobart

In [22]:
Kobart_tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
Kobart = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
Kobart.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [23]:
# input_ids 길이가 1020개 쯔음 이상부터는 에러 발생

s = time.time()
input_ids = Kobart_tokenizer.encode('.'.join(add_title_context))
print('input_ids length: ', len(input_ids))
summary_ids = Kobart.generate(torch.tensor([input_ids[:1020]]).to(device),  num_beams=4,  max_length=512,  eos_token_id=1)
print(Kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))
print(time.time() - s)

input_ids length:  2693
삼성전자는 13 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 밝혔다.
0.4527285099029541


- KoT5보다 짧은 출력이라 사용하기 좋고, 속도도 빠름. 인풋 길이가 길면 에러나는 단점 때문에 여러 요약 단계를 거치거나, 앞부분 기사 내용만 반영해야 하는 단점.

## Test 2
### 기사 제목만 가져다가 요약 생성한 경우

### KoT5

In [24]:
s = time.time()
input_ids = KoT5_tokenizer.encode('.'.join(title_context))
print('input_ids length: ', len(input_ids))
summary_ids = KoT5.generate(torch.tensor([input_ids]).to(device),  num_beams=4,  max_length=512,  eos_token_id=1)
print(KoT5_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))
print(time.time() - s)

input_ids length:  775
삼성전자, 갤럭시 버즈2 프로·워치에 소프트웨어 업데이트.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 갤럭시 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향
11.093422412872314


- 기사 내용 더했을 때 보다 더 오래걸리고 문장도 이상함. 이유는 ??

### Kobart

In [25]:
s = time.time()
input_ids = Kobart_tokenizer.encode('.'.join(title_context))
print('input_ids length: ', len(input_ids))
summary_ids = Kobart.generate(torch.tensor([input_ids[:1020]]).to(device),  num_beams=4,  max_length=512,  eos_token_id=1)
print(Kobart_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))
print(time.time() - s)

input_ids length:  570
삼성전자 버 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 SW 업데이트, 버즈2프로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 줌 인, 워치로 줌 인, 360도 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 SW 업데이트, 버즈2프로 줌 인, 워치로 줌 인, 워치로 스마트폰 카메라 사용성 강화, 버즈2프로 SW 업데이트, 버즈2프로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인, 워치로 줌 인 등 버즈2프로 SW 업데이트 폰카로 360도 사운드
3.9769628047943115


- KoT5랑 비슷하게 시간도 오래걸리고 요약문도 멍청하게 나옴.

## Test 3
### Naver Clova 기사 요약 모델 API

- API 신청: https://www.ncloud.com/product/aiService/clovaSummary
- python 코드 예시: https://blog.naver.com/n_cloudplatform/222485043721

### 제목 + 앞 1,2문장 

In [35]:
client_id = "px9opb5zus"
client_secret = "vBCjpPifA3fmQipwmHs6OS7eBw1D8RsZy6iHC96y"

s = time.time()

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'하루 2000억' 판 커지는 간편송금 시장"
content = '.'.join(add_title_context)[:1500] # 입력 문장이 너무 길면 에러 발생


data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}

# print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code

print(time.time() - s)
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

0.43576884269714355
{"summary":"삼성전자, 버즈2프로 SW 업데이트…360도 소리 녹음 지원 = 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 밝혔습니다.\n스마트폰 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있게 하기 위해서다..삼성전자, 버즈2프로 SW 업데이트…360도 소리 녹음 지원 삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 13일부터 시행합니다.\n삼성전자 버즈2프로 업데이트···\"360도 소리 녹음 지원\" 삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈2 프로 및 갤럭시 Z 플립4·폴드4의 소프트웨어 업데이트를 시행한다고 13일 밝혔습니다."}


- 생성할 요약 문장 개수 지정 가능하고 요약 문장 길이도 짧게 잘 나옴.
- 단점은 높임말로 문장이 생성되어 어색할 수 있다는 점.
- url이랑 title도 같이 넘겨주는데, url은 왜 같이 넘겨주는지 모르겠음.
- 1000회 이하 호출시 -> 무료   //   1000회 초과 호출시 -> 호출당 2원

### 제목만 사용

In [36]:
client_id = "px9opb5zus"
client_secret = "vBCjpPifA3fmQipwmHs6OS7eBw1D8RsZy6iHC96y"

s = time.time()

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

language = "ko" # Language of document (ko, ja )
model = "news" # Model used for summaries (general, news)
tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
summaryCount = "3" # This is the number of sentences for the summarized document.
url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
title= "'하루 2000억' 판 커지는 간편송금 시장"
content = '.'.join(title_context)[:1500] # 입력 문장이 너무 길면 에러 발생


data = {
    "document": {
    "title": title,
    "content" : content
    },
    "option": {
    "language": language,
    "model": model,
    "tone": tone,
    "summaryCount" : summaryCount
    }
}

# print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code

print(time.time() - s)
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

0.33722472190856934
{"summary":"삼성 SW 업데이트.버즈로 360도 녹음, 워치로 줌 인…삼성, 폰카 편의성 높입니다.\n갤럭시 버즈2 프로·워치 등 업데이트, 삼성전자 스마트폰 카메라 사용성 차.삼성전자, 버즈2 프로·워치로 스마트폰 카메라 사용성 한 차원 상향.삼성전자, 버즈2프로 SW 업데이트360도 소리 녹음 지원.삼성전자, 갤럭시버즈2프로 SW 업데이트…360도 소리 녹음 지원.360도 사운드 녹음 삼성전자, 갤럭시 버즈2 프로·워치 SW 업데이트.삼성, 갤버즈2 프로로 플립4·폴드4 360도 녹음 지원.워치로 폰카 줌·버즈로 360도 녹음…삼성전자, 업데이트 진행.녹음되는 갤버즈 vs 보급형 에어팟…삼성-애플, 무선이어폰 대전.버즈로 360도 녹음 '삼성 폰카'의 진화.삼성전자, 버즈2 프로 SW 업데이트 폰카로 360도 사운드 담습니다.\n삼성전자, 갤럭시 버즈2 프로·워치에 소프트웨어 업데이트.삼성전자, 버즈2 프로 SW업데이트360 오디오 레코딩 기능 지원.삼성전자, 버즈2프로·갤워치로 스마트폰 카메라 사용성 높입니다."}


- 기사 내용이 포함되지 않아 구체적인 내용은 담기지 않음.
- 쪼끔 더 빠름